## create Sparse Matrix

fail due to >pd.get_dummies is extremely unefficientcy

In [1]:
import pandas as pd
import numpy as np
X_train = pd.read_csv("data/train.csv")
X_test = pd.read_csv("data/test.csv")

In [2]:
features=X_test.columns.values[2:]

In [3]:
features = X_test.columns.values[2:]
for column in features:
    print("handling col : ",column )
    categories = pd.concat([X_train[column],X_test[column]]).unique()
    X_train[column] = X_train[column].astype('category', categories=categories)
    X_test[column] = X_test[column].astype('category', categories=categories)

handling col :  C1
handling col :  banner_pos
handling col :  site_id
handling col :  site_domain
handling col :  site_category
handling col :  app_id
handling col :  app_domain
handling col :  app_category
handling col :  device_id
handling col :  device_ip
handling col :  device_model
handling col :  device_type
handling col :  device_conn_type
handling col :  C14
handling col :  C15
handling col :  C16
handling col :  C17
handling col :  C18
handling col :  C19
handling col :  C20
handling col :  C21


In [5]:
X_train.to_pickle("Train_cate")
X_test.to_pickle("Test_cate")

In [1]:
import pandas as pd

In [2]:
X_train=pd.read_pickle("Train_cate")
X_test=pd.read_pickle("Test_cate")

In [7]:
# split data into 5 folds
import numpy as np
def n_fold_split(df, k, seed=5566):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df)
    fold_size = int(m/k)
    fold_list = []
    for i in range(k):
        if i!=(k-1):
            target_range=range(i*fold_size,(i+1)*fold_size)
        else:
            target_range=range(i*fold_size,m)
        current_df = df.iloc[perm[target_range]]
        fold_list.append(current_df)
    return fold_list

In [8]:
X_train_list = n_fold_split(X_train,5)
for i in X_train_list:
    print("batchsize:",len(i))

batchsize: 8085793
batchsize: 8085793
batchsize: 8085793
batchsize: 8085793
batchsize: 8085795


In [9]:
features=X_test.columns.values[2:]

In [10]:
features

array(['C1', 'banner_pos', 'site_id', 'site_domain', 'site_category',
       'app_id', 'app_domain', 'app_category', 'device_id', 'device_ip',
       'device_model', 'device_type', 'device_conn_type', 'C14', 'C15',
       'C16', 'C17', 'C18', 'C19', 'C20', 'C21'], dtype=object)

In [ ]:
test = pd.get_dummies(X_test, columns=features, sparse=True)
# extremely slowly and run-out memory (even with only Test dataset)

## build with handly csr matrix
what we want -> a very large matrix with all category variable were translated to dummy

and sloving with logistic regression

### 1. load df from FF ipynb

In [1]:
import pandas as pd 
X_train = pd.read_pickle('train_df_hash')
X_test = pd.read_pickle('test_df_hash')

In [2]:
column2handle = X_train.columns.values[1:]

## 2. split to 5 folds

In [3]:
# split data into 5 folds
import numpy as np
def n_fold_split(df, k, seed=5566):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df)
    fold_size = int(m/k)
    fold_list = []
    for i in range(k):
        if i!=(k-1):
            target_range=range(i*fold_size,(i+1)*fold_size)
        else:
            target_range=range(i*fold_size,m)
        current_df = df.iloc[perm[target_range]]
        fold_list.append(current_df)
    return fold_list

In [4]:
X_train_list = n_fold_split(X_train,5)
for i in X_train_list:
    print("batchsize:",len(i))

batchsize: 8085793
batchsize: 8085793
batchsize: 8085793
batchsize: 8085793
batchsize: 8085795


### 2. filling hash values into dok_matrix

In [5]:
from scipy.sparse import dok_matrix

In [6]:
print("training set size : ",len(X_train_list[0]), "\ntest set size : ",len(X_test))

training set size :  8085793 
test set size :  4577464


In [7]:
X_train_dok = dok_matrix((8085793, 10269200), dtype=np.bool)

In [8]:
%%time
for i in column2handle:
    print(i)
    target = X_train_list[0][i]
    X_train_dok[range(8085793),target.values]=1

C1_hash
banner_pos_hash
site_id_hash
site_domain_hash
site_category_hash
app_id_hash
app_domain_hash
app_category_hash
device_id_hash
device_ip_hash
device_model_hash
device_type_hash
device_conn_type_hash
C14_hash
C15_hash
C16_hash
C17_hash
C18_hash
C19_hash
C20_hash
C21_hash
weekday_hash
hour__hash
CPU times: user 1min 12s, sys: 7.68 s, total: 1min 20s
Wall time: 1min 20s


In [10]:
X_train_dok = X_train_dok.tocsr()

In [12]:
import gc
gc.collect()

7421

In [13]:
# save csr_matrix
from scipy.sparse import csr_matrix

def save_sparse_csr(filename, array):
    # note that .npz extension is added automatically
    np.savez(filename, data=array.data, indices=array.indices,
             indptr=array.indptr, shape=array.shape)

def load_sparse_csr(filename):
    # here we need to add .npz extension manually
    loader = np.load(filename + '.npz')
    return csr_matrix((loader['data'], loader['indices'], loader['indptr']),
                      shape=loader['shape'])

In [14]:
save_sparse_csr('X_train_csr_1', X_train_dok)

In [ ]:
%%time
for j in range(5):
    X_train_dok = dok_matrix(( len(X_train_list[j]), 10269200), dtype=np.bool)
    gc.collect()
    for i in column2handle:
        print(i)
        target = X_train_list[j][i]
        X_train_dok[range(len(X_train_list[j])),target.values]=1
        
    X_train_dok = X_train_dok.tocsr()
    save_sparse_csr('X_train_csr_'+str(j), X_train_dok)
    gc.collect()

In [17]:
range(4,5)

range(4, 5)

## Create SGD Classifier

In [ ]:
SGDClassifier(alpha=.0001, n_iter=50,penalty=penalty))

In [ ]:
def benchmark(clf):
    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = time()
    clf.fit(X_train, y_train)